In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [3]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})

df_48ind = df.iloc[:1171].copy()
df_48ind['Date'] = pd.to_datetime(df_48ind['Date'], format='%Y%m')
df_48ind.set_index('Date', inplace=True)
df_48ind = df_48ind.apply(pd.to_numeric, errors='coerce')
df_48ind.replace(-99.99, np.nan, inplace = True)
df_48ind.replace(-999, np.nan, inplace = True)
df_48ind.dropna(inplace = True)

df_numfirm = df.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')


market_cap = df_numfirm.multiply(df_avgsize, axis=0)
market_cap = market_cap.loc[df_48ind.index]


df_BtoM = df.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

df_BtoM = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

df_BtoM['Date'] = pd.to_datetime(df_BtoM['Date'], format='%Y')

df_BtoM = df_BtoM.drop('Date', axis=1)

df_BtoM.replace(-99.99, np.nan, inplace = True)
df_BtoM.replace(-999, np.nan, inplace = True)
df_BtoM.dropna(inplace = True)

df_BtoM = pd.DataFrame(data = df_BtoM.iloc[5:].values, index = df_48ind.index, columns = df_48ind.columns)


df_mom = df_48ind.rolling(window=12).mean()

df_mom.replace(-99.99, np.nan, inplace = True)
df_mom.replace(-999, np.nan, inplace = True)
df_mom.dropna(inplace = True)


# Créer un objet StandardScaler
scaler = StandardScaler()

def standardize(df):
    # Standardiser les données
    df_scaled = scaler.fit_transform(df)
    
    df_standardized = pd.DataFrame(df_scaled, columns=df.columns, index=df.index)
    
    return df_standardized

standardized_market_caps = standardize(market_cap)
standardized_BtoM = standardize(df_BtoM)
standardized_mom = standardize(df_mom)

market_weights = market_cap.div(market_cap.sum(axis=1), axis=0)


standardized_market_caps = standardized_market_caps.loc[standardized_mom.index]
standardized_BtoM = standardized_BtoM.loc[standardized_mom.index]
standardized_mom = standardized_mom.loc[standardized_mom.index]
market_weights = market_weights.loc[standardized_mom.index]
df_48ind = df_48ind.loc[standardized_mom.index]

df_ret = df_48ind.shift(-1).dropna()

standardized_market_caps = standardized_market_caps.loc[df_ret.index]
standardized_BtoM = standardized_BtoM.loc[df_ret.index]
standardized_mom = standardized_mom.loc[df_ret.index]
market_weights = market_weights.loc[df_ret.index]
df_48ind_ret = df_48ind.loc[df_ret.index]

def calculate_regression_coefficients(N, returns, characteristic1, characteristic2, characteristic3):

    model = LinearRegression()

    beta_initial = []

    for i in range(N):
        y = returns.iloc[:, i]
        X1 = characteristic1.iloc[:, i]
        X2 = characteristic2.iloc[:, i]
        X3 = characteristic3.iloc[:, i]

        X_combined = pd.concat([X1, X2, X3], axis=1)

        model.fit(X_combined, y)
        beta_initial.append(model.coef_)

    return np.mean(beta_initial, axis=0)

coefficients_moyens = calculate_regression_coefficients(df_48ind.shape[1], df_48ind_ret, standardized_market_caps, standardized_BtoM, standardized_mom)


def Expanding_window_optimization(MC, BM, MOM, r, gamma=5):
    
    def u(x, gamma=5):
        return (1+x) ** (1 - gamma) / (1 - gamma)

    def function_to_maximize(beta, MC, BM, MOM, r, T, N, gamma=5):
        wi_t = market_weights
        return -(1/T) * np.sum(u(np.sum((wi_t + (1/N) * (beta[0] * MC + beta[1] * BM + beta[2] * MOM)) * (r),axis=1), gamma))

    optimal_betas = {}  # Define optimal_betas here to store the optimal coefficients for each period
    
    start_year = df_48ind.index.min() # Date de début fixée à la première date dans les données de rendement
    
    # Date de fin initiale fixée à décembre 1973
    initial_end_year = pd.to_datetime('1973-12-01')
    # Date de fin courante, commence par initial_end_year et sera ajustée dans la boucle
    current_end_year = initial_end_year

    # Initial guess for theta
    initial_beta = coefficients_moyens

    while current_end_year <= df_48ind.index.max(): # Boucle jusqu'à la dernière date dans les données de rendement 
        
        # Extraire les composantes nécessaires pour l'optimisation des coefficients pour la période sélectionnée dans la window
        MC = standardized_market_caps.loc[start_year:current_end_year]
        BM = standardized_BtoM.loc[start_year:current_end_year]
        MOM = standardized_mom.loc[start_year:current_end_year]
        r = df_48ind.loc[start_year:current_end_year]
            
        N = df_48ind.shape[1]
        T = df_48ind.shape[0]

        # Run the optimization for the current period
        result = minimize(function_to_maximize, initial_beta, args=( MC, BM, MOM, r, T, N, gamma), method='SLSQP')

        # Store the optimal theta for the current period
        if result.success:
            optimal_betas[current_end_year] = result.x
            initial_beta = result.x  # Update initial_beta with the optimal betas
        else:
            optimal_betas[current_end_year] = None
    

        # Élargir la fenêtre pour la prochaine itération de window en ajoutant 12 mois à la dernière date courante de fin 
        current_end_year += pd.DateOffset(months=12)

    return optimal_betas

In [4]:
market_weights

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1970-06-01,0.000460,0.028814,0.012287,0.005949,0.007738,0.006018,0.003263,0.005133,0.085561,0.005786,...,0.018154,0.018205,0.003409,0.052092,0.005588,0.018704,0.010159,0.001798,0.016279,0.000126
1970-07-01,0.000618,0.030523,0.012310,0.006020,0.008416,0.006368,0.003540,0.005462,0.084660,0.006169,...,0.018282,0.016610,0.004056,0.053393,0.006016,0.021184,0.011817,0.001948,0.016744,0.000137
1970-08-01,0.000568,0.029986,0.012156,0.005991,0.008119,0.006687,0.003318,0.005772,0.082559,0.006217,...,0.018162,0.016776,0.003944,0.055195,0.005586,0.021040,0.012146,0.001964,0.017206,0.000127
1970-09-01,0.000542,0.028717,0.011420,0.005560,0.007744,0.007109,0.003277,0.005859,0.082139,0.006366,...,0.018643,0.017284,0.004015,0.055338,0.005721,0.021386,0.011765,0.001983,0.017315,0.000139
1970-10-01,0.000594,0.028779,0.011321,0.005534,0.007440,0.007211,0.003739,0.006007,0.083506,0.006803,...,0.017829,0.017990,0.004465,0.056529,0.006344,0.020271,0.012227,0.002179,0.017957,0.000141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,0.000995,0.011851,0.007773,0.008085,0.005243,0.000931,0.009142,0.000574,0.015286,0.005949,...,0.001740,0.019324,0.011698,0.074176,0.016307,0.061484,0.035742,0.001602,0.027401,0.026943
2023-09-01,0.000928,0.011433,0.007836,0.007835,0.005182,0.000942,0.008925,0.000620,0.015324,0.005710,...,0.001708,0.018376,0.011755,0.076022,0.015757,0.059433,0.035788,0.001558,0.027243,0.027591
2023-10-01,0.000986,0.011506,0.007677,0.007828,0.005236,0.000919,0.008347,0.000611,0.015038,0.005732,...,0.001748,0.018348,0.012045,0.075141,0.015854,0.060089,0.038577,0.001468,0.027638,0.028226


In [5]:
Expanding_window_optimization(standardized_market_caps, standardized_BtoM, standardized_mom, df_ret)

{Timestamp('1973-12-01 00:00:00'): array([0.54327476, 0.57571289, 2.19551398]),
 Timestamp('1974-12-01 00:00:00'): array([0.39056251, 0.23222941, 2.2230283 ]),
 Timestamp('1975-12-01 00:00:00'): array([0.39711377, 0.22277083, 2.2248954 ]),
 Timestamp('1976-12-01 00:00:00'): array([-77.85352777, 150.85716078,  -0.75936572]),
 Timestamp('1977-12-01 00:00:00'): array([-77.85352777, 150.85716078,  -0.75936572]),
 Timestamp('1978-12-01 00:00:00'): array([-77.85352777, 150.85716078,  -0.75936572]),
 Timestamp('1979-12-01 00:00:00'): array([-77.85352777, 150.85716078,  -0.75936572]),
 Timestamp('1980-12-01 00:00:00'): array([-77.85352777, 150.85716078,  -0.75936572]),
 Timestamp('1981-12-01 00:00:00'): array([-77.85352777, 150.85716078,  -0.75936572]),
 Timestamp('1982-12-01 00:00:00'): array([-77.85352777, 150.85716078,  -0.75936572]),
 Timestamp('1983-12-01 00:00:00'): array([-77.85352777, 150.85716078,  -0.75936572]),
 Timestamp('1984-12-01 00:00:00'): array([-77.85352777, 150.85716078,  -

In [ ]:
def calculate_optimized_weights(mkt_weights, characteristics, optimal_theta):
    optimized_weights = np.zeros(N)
    
    for i in range(N):
        initial_weight = mkt_weights.iloc[i]
        adjustment = sum(optimal_theta[j] * characteristics.iloc[i * M + j] for j in range(M)) / N
        optimized_weights[i] = initial_weight + adjustment

    # Assurer la positivité des poids optimisés
    positive_weights = np.maximum(0, optimized_weights)
    # Normaliser les poids pour que leur somme soit égale à 1 comme dans l'article
    normalized_weights = positive_weights / np.sum(positive_weights)
    
    return normalized_weights

optimized_weights_monthly = {}

start_month = pd.Timestamp('1974-01-01')
filtered_months = mkt_weights[start_month:].index

for current_month in filtered_months:
    current_year = datetime(year=current_month.year, month=12, day=1)
    current_year_str = current_year.strftime('%Y-%m-%d')
    mkt_weights_for_month = mkt_weights.loc[current_month]
    characteristics_for_month = characteristics_standardized_combined.loc[current_month]
    
    # Appliquer les coefficients theta constants à travers les actifs et dans le temps
    optimized_weights = calculate_optimized_weights(mkt_weights_for_month, characteristics_for_month, optimal_thetas[current_year_str])
    
    optimized_weights_monthly[current_month] = optimized_weights

# Convertir le dictionnaire en DataFrame pour une meilleure visualisation
optimized_weights_monthly_df = pd.DataFrame.from_dict(optimized_weights_monthly, orient='index', columns=mkt_weights.columns) optimized_portfolio_returns = (optimized_weights_monthly_df * returns_aligned).sum(axis=1)/100
optimized_portfolio_returns mkt_weights_aligned = mkt_weights.reindex(returns_aligned.index)
market_portfolio_returns = (mkt_weights_aligned * returns_aligned).sum(axis=1)/100

In [ ]:
def calculate_optimized_weights(mkt_weights, MC, BM, mom, optimal_theta):
    optimized_weights = np.zeros(df_48ind.shape[1])
    
    for i in range(df_48ind.shape[1]):
        initial_weight = mkt_weights.iloc[i]
        adjustment = sum((beta1 * MC.iloc[i, :] + beta2 * BM.iloc[i, :] + beta3 * mom.iloc[i, :])) / N
        optimized_weights[i] = initial_weight + adjustment

    # Assurer la positivité des poids optimisés
    positive_weights = np.maximum(0, optimized_weights)
    # Normaliser les poids pour que leur somme soit égale à 1 comme dans l'article
    normalized_weights = positive_weights / np.sum(positive_weights)
    
    return normalized_weights

optimized_weights_monthly = {}

start_month = pd.Timestamp('1974-01-01')
filtered_months = mkt_weights[start_month:].index

for current_month in filtered_months:
    current_year = datetime(year=current_month.year, month=12, day=1)
    current_year_str = current_year.strftime('%Y-%m-%d')
    mkt_weights_for_month = mkt_weights.loc[current_month]
    characteristics_for_month = characteristics_standardized_combined.loc[current_month]
    
    # Appliquer les coefficients theta constants à travers les actifs et dans le temps
    optimized_weights = calculate_optimized_weights(mkt_weights_for_month, characteristics_for_month, optimal_thetas[current_year_str])
    
    optimized_weights_monthly[current_month] = optimized_weights

# Convertir le dictionnaire en DataFrame pour une meilleure visualisation
optimized_weights_monthly_df = pd.DataFrame.from_dict(optimized_weights_monthly, orient='index', columns=mkt_weights.columns) optimized_portfolio_returns = (optimized_weights_monthly_df * returns_aligned).sum(axis=1)/100
optimized_portfolio_returns mkt_weights_aligned = mkt_weights.reindex(returns_aligned.index)
market_portfolio_returns = (mkt_weights_aligned * returns_aligned).sum(axis=1)/100

In [5]:
def Expanding_window_optimization(MC, BM, MOM, r, gamma=5):
    
    def u(x, gamma=5):
        return (1+x) ** (1 - gamma) / (1 - gamma)

    def function_to_maximize(beta, MC, BM, MOM, r, T, N, gamma=5):
        wi_t = market_weights
        return -(1/T) * np.sum(u(np.sum((wi_t + (1/N) * (beta[0] * MC + beta[1] * BM + beta[2] * MOM)) * (r),axis=1), gamma))

    optimal_betas = {}  # Define optimal_betas here to store the optimal coefficients for each period
    
    start_year = df_48ind.index.min() # Date de début fixée à la première date dans les données de rendement
    
    # Date de fin initiale fixée à décembre 1973
    initial_end_year = pd.to_datetime('1973-12-01')
    # Date de fin courante, commence par initial_end_year et sera ajustée dans la boucle
    current_end_year = initial_end_year

    # Initial guess for theta
    initial_beta = coefficients_moyens

    # Define bounds for betas
    beta_bounds = [(0, 5), (0, 5), (0, 5)]

    while current_end_year <= df_48ind.index.max(): # Boucle jusqu'à la dernière date dans les données de rendement 
        
        # Extraire les composantes nécessaires pour l'optimisation des coefficients pour la période sélectionnée dans la window
        MC = standardized_market_caps.loc[start_year:current_end_year]
        BM = standardized_BtoM.loc[start_year:current_end_year]
        MOM = standardized_mom.loc[start_year:current_end_year]
        r = df_48ind.loc[start_year:current_end_year]
            
        N = df_48ind.shape[1]
        T = df_48ind.shape[0]

        # Run the optimization for the current period
        result = minimize(function_to_maximize, initial_beta, args=( MC, BM, MOM, r, T, N, gamma), method='SLSQP', bounds=beta_bounds)

        # Store the optimal theta for the current period
        if result.success:
            optimal_betas[current_end_year] = result.x
            initial_beta = result.x  # Update initial_beta with the optimal betas
        else:
            optimal_betas[current_end_year] = None
    

        # Élargir la fenêtre pour la prochaine itération de window en ajoutant 12 mois à la dernière date courante de fin 
        current_end_year += pd.DateOffset(months=12)

    return optimal_betas

In [6]:
Expanding_window_optimization(standardized_market_caps, standardized_BtoM, standardized_mom, df_ret)

{Timestamp('1973-12-01 00:00:00'): array([1.81006831e-17, 0.00000000e+00, 1.70135439e+00]),
 Timestamp('1974-12-01 00:00:00'): array([2.52560381e-17, 0.00000000e+00, 1.83285236e+00]),
 Timestamp('1975-12-01 00:00:00'): array([0.        , 0.07900923, 1.8529892 ]),
 Timestamp('1976-12-01 00:00:00'): array([0.01593776, 0.18580102, 1.99330336]),
 Timestamp('1977-12-01 00:00:00'): array([0.01593776, 0.18580102, 1.99330336]),
 Timestamp('1978-12-01 00:00:00'): array([0.01593776, 0.18580102, 1.99330336]),
 Timestamp('1979-12-01 00:00:00'): array([0.01593776, 0.18580102, 1.99330336]),
 Timestamp('1980-12-01 00:00:00'): array([0.01593776, 0.18580102, 1.99330336]),
 Timestamp('1981-12-01 00:00:00'): array([0.01593776, 0.18580102, 1.99330336]),
 Timestamp('1982-12-01 00:00:00'): array([0.01593776, 0.18580102, 1.99330336]),
 Timestamp('1983-12-01 00:00:00'): array([0.        , 0.12738134, 2.00198891]),
 Timestamp('1984-12-01 00:00:00'): array([0.       , 0.0649858, 1.7229009]),
 Timestamp('1985-12